In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import warnings

warnings.simplefilter("ignore")

<div class="main-title">
<h1>SRAI</h1>
<p>Design and usage<p>
</div>

## What is SRAI?

<div class="center-content">
    <h3 style="margin: auto;">A toolbox for geospatial AI</h3>
</div>

<div class="center-content">
    <p style="margin: auto;">...that aims to standardize the domain</p>
</div>

<div>
    <img src="assets/SRAI-diagram.png" class="srai-diagram"/>
</div>

Specify the city

In [ ]:
CITY = "Basel"
COUNTRY = "Switzerland"

area_name = f"{CITY}, {COUNTRY}"
area_name

Download the area

In [ ]:
from srai.regionalizers import geocode_to_region_gdf

area = geocode_to_region_gdf(area_name)
area.explore(height=500)

## Loaders

* used to load spatial data from different sources
* unify loading into a single interface
* prepare data for the embedding methods

[API](https://kraina-ai.github.io/srai/latest/api/loaders/)<br>
[Examples](https://kraina-ai.github.io/srai/latest/examples/loaders/)

Types of loaders:

* GeoparquetLoader
* GTFSLoader
* OSMOnlineLoader
* OSMPbfLoader
* OSMWayLoader
* OSMTileLoader

### OSMPbfLoader

* Capable of loading OSM features from a PBF file. 
* It filters features based on OSM tags. 
* Additionally, it can download a pbf file extract for a given area using Protomaps API.

### Let's create a city poster

In [ ]:
from srai.loaders.osm_loaders.filters import BASE_OSM_GROUPS_FILTER

BASE_OSM_GROUPS_FILTER

In [ ]:
from srai.loaders import OSMPbfLoader

features = (
    OSMPbfLoader()
    .load(area, {"highway": True, "water": True, "waterway": True})
    .clip(area)
)
features.head(3)

In [ ]:
features.explore(color="brown")

Plot the poster

In [ ]:
import matplotlib.pyplot as plt
from utils import plot_poster

ax = plot_poster(features, CITY, COUNTRY)

plt.savefig("poster.png", facecolor="#ecedea", dpi=300)
plt.close()

<img src="poster.png" style="height: 800px; margin: auto" /> 

<div>
    <img src="assets/SRAI-diagram.png" class="srai-diagram"/>
</div>

## Regionalizers

* unify methods for dividing a given area into smaller regions. 
* based (mostly) on spatial indexes.

[API](https://kraina-ai.github.io/srai/latest/api/regionalizers/)<br>
[Examples](https://kraina-ai.github.io/srai/latest/examples/regionalizers/)

Types of regionalizers:

* H3Regionalizer
* S2Regionalizer
* VoronoiRegionalizer
* AdministativeBoundaryRegionalizer

### H3Regionalizer

In [ ]:
from srai.regionalizers import H3Regionalizer
from srai.plotting import plot_regions
from utils import CB_SAFE_PALLETE

In [ ]:
regions = H3Regionalizer(resolution=9).transform(area)
plot_regions(regions, colormap=CB_SAFE_PALLETE)

### Data driven regionalization

In [ ]:
from srai.loaders import OSMOnlineLoader

In [ ]:
stops = OSMOnlineLoader().load(area, {"public_transport": "stop_position"})
stops.explore()

In [ ]:
from srai.regionalizers import VoronoiRegionalizer

regionalizer = VoronoiRegionalizer(seeds=stops)
regions = regionalizer.transform(area)

plot_regions(regions, colormap=CB_SAFE_PALLETE)

### Administrative regionalization

In [ ]:
from srai.regionalizers import AdministrativeBoundaryRegionalizer
warnings.simplefilter("ignore")

In [ ]:
regionalizer = AdministrativeBoundaryRegionalizer(admin_level=9)
regions = regionalizer.transform(area)
plot_regions(regions, colormap=CB_SAFE_PALLETE)

In [ ]:
poland = geocode_to_region_gdf("Poland")
regionalizer = AdministrativeBoundaryRegionalizer(admin_level=4)
regions = regionalizer.transform(poland)
plot_regions(regions, colormap=CB_SAFE_PALLETE)

<div>
    <img src="assets/SRAI-diagram.png" class="srai-diagram"/>
</div>